# Disaster Tweets — Basic Network Embeddings & BERT

*Updates: Made some parts of the code neater, and used a different set of validation data.*

This notebook aims to explore the functionality of the [Texthero](https://texthero.org/) package, which makes use of multiple NLP and machine learning toolkits such as Gensim, NLTK, spaCy and scikit-learn. Texthero provides functions to work with text data, and is designed to be used on top of Pandas. We will mainly use Texthero to preprocess text data.

After text cleaning, we attempt to learn our own word embeddings using neural networks for text classification. Following that, we look at transfer learning through the use of pre-trained models. There are many pre-trained word embeddings like [GloVe](https://nlp.stanford.edu/projects/glove/), but we look at a more robust contextual embedding, [BERT](https://github.com/google-research/bert), which generates embeddings that takes into account the context or surrounding words, and is based on the Transformer architecture.

In [ ]:
!pip install texthero spaCy==3.3.0 gensim==3.8.3

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import texthero as hero
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_classif
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping
from tensorflow_addons.metrics import F1Score

plt.style.use("fivethirtyeight")

---

# 1. Exploring Data

In [ ]:
train = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv").drop(columns=["location"])
test = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv").drop(columns=["location"])

print("="*40)
print(f"Dimensions of training data: {train.shape}")
print(f"Dimensions of testing data: {test.shape}")
print("="*40)
train.tail()

For the problem to identify disaster tweets, we will not be using the location column. While the column may convey information on the physical location of the disaster, we can check that there are way too many unique values (almost half the training data) for the column to be useful.

In contrast, the keyword column only has 222 unique values and may provide additional information that is not present in the tweet. For simplicity, we fill the missing values with an empty string to indicate the lack of a keyword. From the percentage plot below, we observe that there are many keywords which mostly belong to disaster or non-disaster tweets. 

However, we note that there are also a number of keywords with approximately equal proportions in both disaster and non-disaster tweets. These keywords may not be as helpful for our models. To identify the useful keywords, we need to set a threshold. Before that, we need to clean the data because some keywords like ``suicide%20bombing`` and ``suicide%20bomb`` generally refer to the same context. We look at text cleaning in the next section.

In [ ]:
def plot_top_keywords(df_train, top=15, mid=0, **kwargs):
    assert top!=0 and mid>=0. and mid<=1.
    df = df_train.copy()
    df_count = df.groupby("keyword").target.value_counts().unstack("target").fillna(0).astype(int)
    df_prop = df.groupby("keyword").target.value_counts(normalize=True).unstack("target").fillna(0).sort_values(0)
    if not mid:
        df_plot = df_prop.iloc[:top, :].iloc[::-1] if top>0 else  df_prop.iloc[top:, :]
    else:
        idx = int(mid*len(df_prop.index))
        df_plot = df_prop.iloc[idx:idx+top, :]
    
    df_plot.plot(
        kind="barh",
        stacked=True,
        **kwargs,
    )
    for idx, row in df_plot.iterrows():
        if row[0] > 0.2:
            plt.text(
                x=row[0]/2,
                y=df_plot.index.tolist().index(idx),
                s=f"{row[0]*100:.1f}% ({df_count.loc[idx, 0]})",
                va="center",
                ha="center",
                color="white",
            )
        if row[1] > 0.2:
            plt.text(
                x=row[0]+row[1]/2,
                y=df_plot.index.tolist().index(idx),
                s=f"{row[1]*100:.1f}% ({df_count.loc[idx, 1]})",
                va="center",
                ha="center",
                color="white",
            )
    
    num_keyword = len(df_plot.index)
    if not mid:
        top_or_bot = "Highest" if top>0 else "Lowest"
        plt.title(f"Top {num_keyword} Keywords with {top_or_bot} % of Disaster Tweets")
    else:
        plt.title(f"Random {num_keyword} Keywords")
        
    plt.xlabel("Proportion")
    plt.ylabel("")
    plt.legend(["Not Disaster", "Disaster"], loc="upper left")

In [ ]:
train.keyword = train.keyword.fillna("")
test.keyword = test.keyword.fillna("")

fig = plt.figure(figsize=(18, 18))

plot_top_keywords(train, top=15, ax=fig.add_subplot(221))
plot_top_keywords(train, top=-15, ax=fig.add_subplot(222))
plot_top_keywords(train, mid=0.35, ax=fig.add_subplot(223))

plt.tight_layout()

---

# 2. Understanding Text Preprocessing

We now create functions for text cleaning. Our data contains the following non-exhaustive list of information:
* Uppercased and lowercased characters,
* Diacritics and accents,
* Punctuation marks,
* Blocks of digits,
* Stopwords,
* Words of different forms (e.g. bags vs. bag),
* Additional whitespaces,
* URLs,
* HTML tags,
* Tweet @mentions.

There could be other information like emojis and spelling errors, but we will make do with the above list, which can still be managed using Texthero methods. The list of methods can be found [here](https://texthero.org/docs/api-preprocessing) and [here](https://github.com/jbesomi/texthero/blob/master/texthero/preprocessing.py), or by simply calling the ``dir()`` function. 

In [ ]:
print(dir(hero))

Depending on the context of the problem, it may be useful to standardize/remove the above information, or leave it as is. To determine the usefulness of the information, we temporarily replace them with tags (e.g. ``[PTT]`` for punctuation marks). After text cleaning, we can extract features from these tags. The functions defined for text cleaning are listed below.

In [ ]:
def text_pp(s, fn):
    print("BEFORE:")
    print(s.values[0])
    print("\nAFTER:")
    print(fn(s).values[0])

### Coverting to lowercase.

In [ ]:
text_pp(train[["text"]].iloc[210], fn=hero.lowercase)

### Removing diacritics and accents.

In [ ]:
text_pp(train[["text"]].iloc[38], fn=hero.remove_diacritics)

### Replacing URLs.

In [ ]:
def replace_urls(s):
    return hero.replace_urls(s, symbol=" [URL] ")

In [ ]:
text_pp(train[["text"]].iloc[-1], fn=replace_urls)

### Replacing HTML tags.

In [ ]:
def replace_html_tags(s):
    pattern = r"""(?x)                              
      <[^>]+>                                       
      | &([a-z0-9]+|\#[0-9]{1,6}|\#x[0-9a-f]{1,6});
      """
    
    return s.str.replace(pattern, "[HTM]", regex=True)

In [ ]:
text_pp(train[["text"]].iloc[64], fn=replace_html_tags)

### Replacing @mentions.

In [ ]:
def replace_mentions(s):
    return s.str.replace(r"@\w+", " [MTN] ", regex=True)

In [ ]:
text_pp(train[["text"]].iloc[54], fn=replace_mentions)

### Replacing punctuations.

In [ ]:
def replace_punctuation(s):
    s = s.str.replace(r"[^a-zA-Z0-9\s\[\]]", " [PTT] ", regex=True)
    s = s.str.replace(r"\[(?![A-Z]{3}\])", " [PTT] ", regex=True)
    s = s.str.replace(r"(?<!\[[A-Z]{3})\]", " [PTT] ", regex=True)
    
    return s

In [ ]:
text_pp(train[["text"]].iloc[2641], fn=replace_punctuation)

### Replacing digits.

In [ ]:
def replace_digits(s):
    return hero.replace_digits(s, symbols=" [DIG] ")

In [ ]:
text_pp(train[["text"]].iloc[45], fn=replace_digits)

### Replacing stopwords.

In [ ]:
def replace_stopwords(s):
    return hero.replace_stopwords(s, symbol=" [STP] ")

In [ ]:
text_pp(train[["text"]].iloc[18], fn=replace_stopwords)

### Stemming words.

In [ ]:
text_pp(train[["text"]].iloc[14], fn=hero.stem)

### Removing whitespaces.

In [ ]:
text_pp(train[["text"]].iloc[7552], fn=hero.remove_whitespace)

In [ ]:
def remove_spaces(s):
    return s.str.replace(r"%20", " ", regex=True)

In [ ]:
text_pp(train[["keyword"]].iloc[136], fn=remove_spaces)

---

# 3. Defining Custom Pipelines for Text Preprocessing

### Cleaning keywords.

We first clean the keyword column before combining it with the actual tweet. We define a custom pipeline containing the relevant methods used for our problem; the functions are applied according to the order listed in the pipeline.

After text cleaning, the number of unique keywords reduced from 222 to 167, indicating that many of the original keywords listed share similar information but are labeled in a different form. As described earlier, we set the threshold such that keywords with at least 70% disaster or non-disaster tweets will be combined with the original tweet.

In [ ]:
clean_keywords = [
    remove_spaces,
    hero.remove_stopwords,
    hero.stem,
    hero.remove_whitespace,
]

train["clean_keyword"] = hero.clean(train.keyword, pipeline=clean_keywords)
test["clean_keyword"] = hero.clean(test.keyword, pipeline=clean_keywords)

print("="*40)
print(f"{train.keyword.nunique()} keywords reduced to {train.clean_keyword.nunique()} keywords:")
print("="*40)
print(train.clean_keyword.unique().tolist())

In [ ]:
THRESH = 0.7

keyword_prop = train.groupby("clean_keyword").target.value_counts(normalize=True).unstack("target")
keyword_prop = keyword_prop.fillna(0).sort_values(0)
keywords_keep = keyword_prop[(keyword_prop[0]<=(1-THRESH)) | (keyword_prop[0]>=THRESH)].index

train.clean_keyword = train.clean_keyword.apply(lambda x: x if x in keywords_keep else "")
test.clean_keyword = test.clean_keyword.apply(lambda x: x if x in keywords_keep else "")

### Cleaning text.

 With the large amount of information to be cleaned in the actual tweet, the order of the functions listed in the pipeline is important. For example, if we replace punctuations before replacing URLs, the forward slashes ``/`` contained in URLs will be removed so the original URLs may not be recognized properly. Subsequently, we combine the keywords with the tweets by adding the keywords to the end.

In [ ]:
clean_text = [
    hero.lowercase,
    hero.remove_diacritics,
    hero.remove_whitespace,
    replace_urls,
    replace_html_tags,
    replace_mentions,
    replace_punctuation,
    replace_digits,
    replace_stopwords,
    hero.stem,
    hero.remove_whitespace,
]

train["clean_text"] = train.text + " " + train.clean_keyword
test["clean_text"] = test.text + " " + test.clean_keyword

train.clean_text = hero.clean(train.clean_text, pipeline=clean_text)
test.clean_text = hero.clean(test.clean_text, pipeline=clean_text)

train.head()

---

# 4. Feature Engineering

### Extracting features from text.

After text cleaning, we can extract features from the text by counting the tags used to temporarily replace the corresponding information. In addition, other features like counting the number of characters or number of words (excluding the information removed) can be created.

In [ ]:
def extract_features(df):
    if any("num" in j for j in df.columns):
        return df
    df = df.copy()
    
    features = ["url", "htm", "mtn", "ptt", "dig", "stp"]
    for feat in features:
        df[f"num_{feat}"] = df.clean_text.apply(lambda x: x.count(f"[{feat}]"))
        df.clean_text = df.clean_text.str.replace(rf"\[{feat}\]", "", regex=True)
        
    df.clean_text = hero.remove_whitespace(df.clean_text)
    df["num_char"] = (df.clean_text).apply(len)
    df["num_word"] = hero.tokenize(df.clean_text).apply(len)
    
    return df

In [ ]:
train = extract_features(train)
test = extract_features(test)

train.head()

### Evaluating usefulness of features extracted.

To evaluate the usefulness of the features extracted, we look at their density plots. For a better quantification, we can estimate the mutual information, measuring the dependency between the features and the target variable. We observe that the only difference in distributions is that disaster tweets tend to contain URLs (where values are most concentrated on 1 and above), and have slightly more characters and words (where the distribution is shifted a little to the right). Otherwise, the distributions for both disaster and non-disaster tweets look similar for most features. For simplicity, we do not consider any of these features in our models.

In [ ]:
def feature_plot(df, features):
    df = df.copy()
    
    nrow = (len(features)//3) + 1
    fig = plt.figure(figsize=(18, 4.5*nrow))
    for i, feat in enumerate(features):
        ax = fig.add_subplot(nrow, 3, i+1)
        
        sns.kdeplot(
            x=df[feat][df.target==0],
            fill=True,
            lw=3,
            ax=ax,
            label="Not Disaster",
        )
        # plot separately to rescale for better comparison of the distribution shape
        sns.kdeplot(
            x=df[feat][df.target==1],
            fill=True,
            lw=3,
            ax=ax,
            label="Disaster",
        )
        
        mi = mutual_info_classif(
            X=df[feat].values.reshape(-1,1),
            y=df.target,
            random_state=1234,
        )[0]
        ax.set_title(f"Mutual Information: {mi:.4f}")
        ax.legend(loc="upper right")
        
    plt.tight_layout()

In [ ]:
feature_plot(train, features=[col for col in train.columns if "num" in col])

### Detecting *n*-grams.

*N*-grams are defined to be a contiguous sequence of *n* words. Because we are dealing with tweets which are generally informal and short in length, we only consider up to bigrams. We use ``hero.PhrasesTransformer()`` which wraps ``gensim.models.phrases.Phrases`` to detect bigrams based on collocation counts. The two words are concatenated together if they are detected as bigrams.

In [ ]:
train_tokenized = hero.tokenize(train.clean_text)
test_tokenized = hero.tokenize(test.clean_text)

bigram_detector = hero.PhrasesTransformer()
train_bigram = bigram_detector.fit_transform(train_tokenized)
test_bigram = bigram_detector.transform(test_tokenized)

train["bigram_text"] = pd.Series(train_bigram).apply(lambda x: " ".join(x))
test["bigram_text"] = pd.Series(test_bigram).apply(lambda x: " ".join(x))

Using the example shown below, the words ``heavy`` and ``flash`` belonging to ``heavy rain`` and ``flash flood`` could have totally different meanings by themselves depending on the context.

In [ ]:
print("BEFORE:")
print(train_tokenized[6])
print("\nAFTER:")
print(train_bigram[6])

---

# 5. Classifying Text with Basic Network Embeddings

In [ ]:
train_data, valid_data, train_labels, valid_labels = train_test_split(
    train.drop(columns=["target"]), train.target,
    test_size=0.1,
    stratify=train.target,
    random_state=1234,
)

### Tokenizing text.

To feed text into neural networks we need to tokenize the text and then index or vectorize them. Here we are trying to learn our own word embeddings using an embedding layer, so we turn each text into a sequence of indices using ``tensorflow.keras.preprocessing.text.Tokenizer``, each unique word having its own index. We can specify the size of the vocabulary ``VOCAB_SIZE``, denoting the maximum number of words to keep based on word frequency. To ensure our text inputs are of the same size, we call ``tensorflow.keras.preprocessing.sequence.pad_sequences`` to pad or truncate each sequence to the same length. This length can be controlled by ``MAX_LENGTH``. Also, there could be words present in the validation or testing data that are not found in the training data. To allow for better learning, these missing words can be represented using a out-of-vocabulary (OOV) token ``OOV_TOK``.

In [ ]:
VOCAB_SIZE = 10000
MAX_LENGTH = 20
OOV_TOK = "[OOV]"

tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token=OOV_TOK, filters="")
tokenizer.fit_on_texts(train_data.bigram_text)

train_seq = tokenizer.texts_to_sequences(train_data.bigram_text)
train_padded = pad_sequences(train_seq, maxlen=MAX_LENGTH)

valid_seq = tokenizer.texts_to_sequences(valid_data.bigram_text)
valid_padded = pad_sequences(valid_seq, maxlen=MAX_LENGTH)

test_seq = tokenizer.texts_to_sequences(test.bigram_text)
test_padded = pad_sequences(test_seq, maxlen=MAX_LENGTH)

Below is an example of the results from tokenization, where words like ``previouslyondoyintv``, ``makinwaua`` and ``marriag`` are not found in the training data and are thus replaced by the OOV token. By default, padding and truncating happens at the start of the sequence.

In [ ]:
print("ORIGINAL CLEAN TEXT:")
print(test.bigram_text.iloc[17])
print("\nTOKENIZED SEQUENCE:")
print(test_seq[17])
print("\nPADDED SEQUENCE:")
print(test_padded[17].tolist())
print("\nTRANSFORMED TEXT:")
print(tokenizer.sequences_to_texts([test_seq[17]])[0])

### Setting model configuration parameters.

For model training, we set up an exponential decay learning rate schedule and early stopping to prevent overfitting.

In [ ]:
INIT_LR = 5e-4
DECAY_RATE = 0.5

lr_schedule = LearningRateScheduler(lambda epoch: INIT_LR * (DECAY_RATE**epoch))
early_stopping = EarlyStopping(
    monitor="val_loss",
    min_delta=1e-4, # minor improvements not considered
    patience=3,
    restore_best_weights=True,
)
metric = F1Score(num_classes=1, threshold=0.5)

MODEL_CONFIG = {
    "optimizer": "adam",
    "loss": "binary_crossentropy",
    "metrics": [metric],
}
TRAIN_CONFIG = {
    "x": train_padded,
    "y": train_labels,
    "callbacks": [lr_schedule, early_stopping],
    "epochs": 50, # training stopped using early stopping
    "batch_size": 16,
    "validation_data": (valid_padded, valid_labels),
    "verbose": 0,
}

def model_predict(model, X):
    return (model.predict(X, verbose=0)>0.5).reshape(-1,).astype(int)

In [ ]:
def model_eval(history, true, pred):
    num_epoch = len(history["loss"])
    xtick_freq = (num_epoch//10) + 1
    xtick_labels = np.arange(num_epoch, step=xtick_freq, dtype=int)
    plt.figure(figsize=(18, 4.5))

    plt.subplot(131)
    plt.plot(history["loss"], "-o", label = "Training")
    plt.plot(history["val_loss"], "-o", label = "Validation")
    plt.xlabel("Epoch")
    plt.xticks(xtick_labels, xtick_labels+1)
    plt.ylabel("Loss")
    plt.title("Loss Curves")
    plt.legend(loc="upper right")

    plt.subplot(132)
    plt.plot(history["f1_score"], "-o", label = "Training")
    plt.plot(history["val_f1_score"], "-o", label = "Validation")
    plt.xlabel("Epoch")
    plt.xticks(xtick_labels, xtick_labels+1)
    plt.ylabel("F1 Score")
    plt.title("F1 Score Curves")
    plt.legend(loc="lower right")

    plt.subplot(133)
    idx = np.argmin(history["val_loss"])
    score = history["val_f1_score"][idx][0]
    cf_mat = confusion_matrix(true, pred)
    cf_mat_norm = confusion_matrix(true, pred, normalize="all")
    labels = [f"{ct}\n({pt*100:.1f}%)" for ct, pt in zip(cf_mat.ravel(), cf_mat_norm.ravel())]
    labels = np.array(labels).reshape(2, 2)

    sns.heatmap(
        cf_mat,
        annot=labels,
        cmap="Blues",
        fmt="",
    )

    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.title(f"Validation F1 Score: {score:.4f}")
    
    plt.tight_layout()    

### Building network architecture.

Neural network embeddings provide a method to represent discrete variables as continuous vectors. Here we are trying to learn continuous vector representations of each word. This helps reduce dimensionality, compared to other methods like one-hot embedding which maps each word to a different vector. Moreover, the vectors of words that share similar meaning in the given context will be closer in the embedding space.

To build the model, we first define an embedding layer that turns the text input into vectors of dimensions specified by ``output_dim``. As this is a binary classification problem, the output layer has simply 1 unit, with a sigmoid activation function mapping the output to a probability value. We look at using different layers after the embedding layer:
* **Bidirectional GRU** — connects 2 GRU layers of opposite directions to the next layer. This has the advantage of getting information from past and future states simultaneously, effectively understanding the words in context.
* **Convolution + global average pooling**. The convolution layer extracts features while the global average pooling layer condenses the information of each feature map.

For simplicity, we do not add additional layers, and the models are manually tuned. The results are shown below.

In [ ]:
def build_gru():
    gru_model = tf.keras.Sequential([
        tf.keras.layers.Embedding(VOCAB_SIZE, output_dim=16, input_length=MAX_LENGTH),
        tf.keras.layers.Bidirectional(tf.keras.layers.GRU(16, dropout=0.4)),
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ])
    gru_model.compile(**MODEL_CONFIG)
    
    return gru_model

def build_cnn():
    cnn_model = tf.keras.Sequential([
        tf.keras.layers.Embedding(VOCAB_SIZE, output_dim=16, input_length=MAX_LENGTH),
        tf.keras.layers.Conv1D(128, kernel_size=3, activation="relu", padding="same"),
        tf.keras.layers.GlobalAveragePooling1D(),
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ])
    cnn_model.compile(**MODEL_CONFIG)
    
    return cnn_model

### Training GRU model.

In [ ]:
gru_model = build_gru()
gru_history = gru_model.fit(**TRAIN_CONFIG).history

model_eval(gru_history, valid_labels, model_predict(gru_model, valid_padded))

### Training CNN model.

In [ ]:
cnn_model = build_cnn()
cnn_history = cnn_model.fit(**TRAIN_CONFIG).history

model_eval(cnn_history, valid_labels, model_predict(cnn_model, valid_padded))

---

# 6. Classifying Text with BERT

There are many different BERT models that can be loaded from [TensorFlow Hub](https://tfhub.dev/) together with their corresponding preprocessing models. We can try different models by simply changing the below URLs to be loaded from.

In [ ]:
HANDLE_PREPROCESS = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
HANDLE_ENCODER = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/4"

### Understanding preprocessing model.

There are 3 outputs from preprocessing, which would then be used as inputs for the BERT model: ``input_word_ids`` is the unique index for each tokenized input, ``input_mask`` differentiates non-padding from padding tokens, and ``input_type_ids`` distinguishes multiple text segments per input. The input is truncated to 128 tokens by default.

In [ ]:
bert_preprocessor = hub.KerasLayer(HANDLE_PREPROCESS)
example_input = bert_preprocessor([train.text.values[0]])

print("input_word_ids: ", example_input["input_word_ids"].numpy().tolist())
print("\ninput_mask    : ", example_input["input_mask"].numpy().tolist())
print("\ninput_type_ids: ", example_input["input_type_ids"].numpy().tolist())

### Understanding BERT model.

The BERT model also returns 3 outputs: ``pooled_output`` can be regarded as the embedding for each tweet, ``sequence_output`` can be considered the embedding for every token in each tweet, and ``encoder_outputs`` are the intermediate activations of the Transformer blocks. We are mainly concerned with ``pooled_output`` for our model.

In [ ]:
bert_encoder = hub.KerasLayer(HANDLE_ENCODER)
example_output = bert_encoder(example_input)

print("pooled_output shape  : ", example_output["pooled_output"].shape)
print("sequence_output shape: ", example_output["sequence_output"].shape)
print(f"encoder_outputs shape: {len(example_output['encoder_outputs'])} Tensors with shape", example_output["encoder_outputs"][0].shape)

### Building network architecture.

We build a [simple model](https://www.tensorflow.org/text/tutorials/classify_text_with_bert#define_your_model) using only the preprocessing model, the BERT model, and the output layer.

In [ ]:
def build_bert():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
    bert_preprocessor = hub.KerasLayer(HANDLE_PREPROCESS)
    encoder_inputs = bert_preprocessor(text_input)
    bert_encoder = hub.KerasLayer(HANDLE_ENCODER, trainable=True)
    outputs = bert_encoder(encoder_inputs)
    net = outputs["pooled_output"]
    net = tf.keras.layers.Dense(1, activation="sigmoid")(net)
    
    bert_model = tf.keras.Model(text_input, net)
    bert_model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
        loss="binary_crossentropy",
        metrics=[metric],
    )
    
    return bert_model

### Training BERT model.

In [ ]:
bert_model = build_bert()

early_stopping.patience = 1 # training stopped if current epoch shows no improvement
bert_history = bert_model.fit(
    x=train_data.text, # original text column used
    y=train_labels,
    epochs=50, # training stopped using early stopping
    callbacks=[early_stopping],
    batch_size=16,
    validation_data=(valid_data.text, valid_labels),
).history

In [ ]:
model_eval(bert_history, valid_labels, model_predict(bert_model, valid_data.text))

---

# 7. Submission

The pre-trained BERT model achieved a much better performance, showing the benefits of transfer learning. We use it as the final model for prediction.

In [ ]:
submission = pd.DataFrame({
    "id": test.id,
    "target": model_predict(bert_model, test.text),
})

submission.head()

In [ ]:
submission.to_csv("submission.csv", index=False)

---